In [2]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
import seaborn as sns
import matplotlib.pyplot as plt
from math import sqrt
from scipy import stats

from env import get_db_url
import acquire

In [ ]:
sql_all = acquire.sql_all

In [ ]:
zillow = acquire.get_data_from_sql(sql_all)

In [ ]:
zillow.columns.tolist()

In [ ]:
zillow.calculatedfinishedsquarefeet.isna().sum()

In [ ]:
# Drop all rows with nan sq_ft
zillow = zillow.dropna(subset=['calculatedfinishedsquarefeet'])

In [ ]:
zillow.calculatedfinishedsquarefeet.isna().sum()

In [ ]:
# Replace NaNs with 0
zillow = zillow.fillna(0)

In [ ]:
# Consolidate airconditioningtypeid into Has_AC = 1, else 0
zillow['has_AC'] = np.where(zillow.airconditioningtypeid > 0, '1', '0')
# Consolidate garage into has_garage
zillow['has_garage'] = np.where(zillow.garagecarcnt > 0, '1', '0')
# Consolidate extra features like fireplace, basement, pool etc into one column
zillow['is_extra'] = np.where(zillow.basementsqft > 0, '1',
                              np.where(zillow.fireplaceflag > 0, '1',
                                       np.where(zillow.poolcnt > 0, '1',
                                                np.where(zillow.decktypeid > 0, '1',
                                                         np.where(zillow.hashottuborspa > 0, '1', '0')))))

In [ ]:
# Determine how many rows were generated by these consolidations
zillow.has_AC.value_counts()

In [ ]:
zillow.has_garage.value_counts()

In [ ]:
zillow.is_extra.value_counts(), 

In [ ]:
pd.crosstab(zillow.has_AC, zillow.is_extra)

In [ ]:
zillow.head()

In [ ]:
# Drop columns I replaced with consolidation
zillow = zillow.drop(columns=[
    'airconditioningtypeid', 'basementsqft', 'fireplaceflag', 'fireplacecnt', 'poolcnt', 
    'decktypeid', 'hashottuborspa', 'garagecarcnt', 'garagetotalsqft', 
    'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7'
                             ])

# Drop columns I can't use
zillow = zillow.drop(columns=[
    'architecturalstyletypeid', 'id', 'buildingclasstypeid', 'buildingqualitytypeid', 'calculatedbathnbr',
    'assessmentyear', 'taxdelinquencyflag', 'taxdelinquencyyear', 'censustractandblock', 'logerror', 
    'finishedfloor1squarefeet', 'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
    'finishedsquarefeet50', 'finishedsquarefeet6', 'fullbathcnt', 'heatingorsystemtypeid',
    'latitude', 'longitude', 'propertycountylandusecode', 'propertylandusetypeid', 'propertyzoningdesc',
    'rawcensustractandblock','storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid', 'unitcnt',
    'yardbuildingsqft17', 'yardbuildingsqft26', 'transactiondate', 'regionidcity', 'regionidcounty', 'regionidzip', 
                             ])

In [ ]:
zillow.head()

In [ ]:
zillow.fips.value_counts()

In [ ]:
# Change the fips column per specs
zillow.fips = zillow.fips.astype(int)
zillow['county'] = np.where(zillow.fips == 6037, 'Los_Angeles',
                           np.where(zillow.fips == 6059, 'Orange', 
                                   'Ventura'))
zillow['state'] = 'CA'

In [ ]:
# Make new categorical columns for what county properties are located in
zillow['is_LA'] = np.where(zillow.fips == 6037, '1', '0')
zillow['is_Or'] = np.where(zillow.fips == 6059, '1', '0')
zillow['is_Ven'] = np.where(zillow.fips == 6111, '1', '0')

In [ ]:
# Drop the fips column, as it has outlived its usefulness
zillow = zillow.drop(columns='fips')

In [ ]:
# Add tax rate column per specs, and drop tax columns from data (because cheating)
zillow['tax_rate'] = (zillow.taxamount / zillow.taxvaluedollarcnt).round(4)
zillow = zillow.drop(columns=['structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount'])

In [ ]:
# Because my model had such low correlation between bedroom and bathroom counts,
# I'm going to add them together and make a new column
zillow['bath_bed'] = zillow.bathroomcnt + zillow.bedroomcnt

In [ ]:
# Calulate the age of the property
zillow['age'] = 2017 - zillow.yearbuilt

In [ ]:
# Calulate ratio of home size to lot size
# first drop all rows with 0 sq_ft

zillow['size_ratio'] = (zillow.calculatedfinishedsquarefeet / zillow.lotsizesquarefeet).round(4)

In [ ]:
zillow.head()

In [ ]:
zillow.numberofstories.value_counts()

In [ ]:
# Replace 0s in numberofstories with 1
zillow.numberofstories = zillow.numberofstories.replace([0], 1)

In [ ]:
# divide avg sq feet of a neighborhood by (number of houses / avg sq feet) 
thing = pd.DataFrame(zillow.regionidneighborhood.value_counts())
thing = thing.rename(columns={'regionidneighborhood': 'num_houses'})
thing['avg_sq_ft'] = zillow.groupby('regionidneighborhood')['calculatedfinishedsquarefeet'].agg('median')
thing['factor'] = thing.avg_sq_ft / (thing.num_houses / thing.avg_sq_ft)

thing

In [ ]:
# apply factor from thing dataframe into the zillow dataframe
#zillow['factor'] = np.where(zillow.regionidneighborhood == thing.index, thing.factor)


In [ ]:
zillow.head()

In [ ]:
# Rename columns to be more friendly
zillow = zillow.rename(columns={
    "calculatedfinishedsquarefeet": "home_sf",
    "lotsizesquarefeet": "lot_sf",
    "numberofstories": "stories",
    "taxvaluedollarcnt": "value",
})

In [ ]:
zillow.head()

In [ ]:
# Now lets see how these columns correlate
plt.figure(figsize=(20,10))
corr = zillow.corr()
ax = sns.heatmap(corr, annot=True, cmap="YlGnBu")
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

In [ ]:
# Looking at the value column, it looks like bathroomcnt, home_sf are well correlated. 
# The engineered column bath_bed is better correlated than most, 
# but not as well as just bathroom count, so I'll drop it
# Same for the engineered column size_ratio, but since it is better correlated than lot_sf, 
# I'll leave it and drop lot_sf

# Negatively correlated columns will be dropped:
# roomcnt, yearbuilt, tax_rate
# Low correlation columns will be dropped:
# stories, age

In [ ]:
# Run t-test between value and categorical variables
# has_AC, has_garage, is_extra, is_LA, is_Or, is_Ven
# As the difference between the sample data and the null hypothesis increases,
# the absolute value of the t-value increases. So, higher t stat is better
x1 = zillow[zillow.has_AC == '1'].value
x2 = zillow[zillow.has_AC == '0'].value

stats.ttest_ind(x1, x2)

In [ ]:
x1 = zillow[zillow.has_garage == '1'].value
x2 = zillow[zillow.has_garage == '0'].value

stats.ttest_ind(x1, x2)

In [ ]:
x1 = zillow[zillow.is_extra == '1'].value
x2 = zillow[zillow.is_extra == '0'].value

stats.ttest_ind(x1, x2)

In [ ]:
x1 = zillow[zillow.is_LA == '1'].value
x2 = zillow[zillow.is_LA == '0'].value

stats.ttest_ind(x1, x2)

In [ ]:
x1 = zillow[zillow.is_Or == '1'].value
x2 = zillow[zillow.is_Or == '0'].value

stats.ttest_ind(x1, x2)

In [ ]:
x1 = zillow[zillow.is_Ven == '1'].value
x2 = zillow[zillow.is_Ven == '0'].value

stats.ttest_ind(x1, x2)

In [ ]:
# it looks like is_extra is the best of all, and location isn't very relevant.
# added neighborhood column to check if location helps

In [ ]:
# Remaining columns for model:
# y = value
# X = home_sf, bathroomcnt, bedroomcnt, size_ratio, is_extra

In [11]:
# Test new wrangle_zillow_FE function
sql_FE = acquire.sql_FE
zillow_FE = acquire.wrangle_zillow_FE(sql_FE)
zillow_FE.head()

,parcelid,neighborhood,value,baths,beds,home_sf,is_extra,county,state,tax_rate,size_ratio
0,11289917,0,136104,2.000,3,1458,1,Los_Angeles,CA,0.017,0.176
1,11705026,118208,35606,1.000,2,1421,0,Los_Angeles,CA,0.015,0.212
2,14269464,0,880456,3.000,4,2541,0,Orange,CA,0.011,0.511
3,11389003,0,614000,2.000,3,1650,0,Los_Angeles,CA,0.013,0.226
4,11967869,116774,274237,1.000,2,693,0,Los_Angeles,CA,0.012,0.238


In [12]:
zillow_FE.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14912 entries, 0 to 15035
Data columns (total 11 columns):
parcelid        14912 non-null int64
neighborhood    14912 non-null int64
value           14912 non-null int64
baths           14912 non-null float64
beds            14912 non-null int64
home_sf         14912 non-null int64
is_extra        14912 non-null object
county          14912 non-null object
state           14912 non-null object
tax_rate        14912 non-null float64
size_ratio      14912 non-null float64
dtypes: float64(3), int64(5), object(3)
memory usage: 1.4+ MB


In [13]:
zillow_FE.neighborhood.value_counts()

0         9461
48570      138
27080      130
118208     121
113455     113
          ... 
761215       1
761223       1
267814       1
761543       1
416303       1
Name: neighborhood, Length: 370, dtype: int64

In [14]:
pd.set_option('use_inf_as_na', True)

In [15]:
zillow_FE.isna().sum()

parcelid        0
neighborhood    0
value           0
baths           0
beds            0
home_sf         0
is_extra        0
county          0
state           0
tax_rate        0
size_ratio      0
dtype: int64

In [18]:
np.isinf(zillow_FE.size_ratio).sum()

0

In [19]:
# Save zillow_FE as csv
zillow_FE.to_csv('zillow_FE.csv')